In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, cos, date_format, lit, sin, sum, when, to_date, to_timestamp, hour

spark = SparkSession.builder.appName("coffee").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/04 20:08:13 WARN Utils: Your hostname, mirs-pc resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/04 20:08:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 20:08:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load & Check the sales DataFrames
df_sales = spark.read.csv("./data/Coffee Shop Sales.csv", header=True)
df_sales.head(5)

[Row(transaction_id='1', transaction_date='1/1/2023', transaction_time='7:06:11', transaction_qty='2', store_id='5', store_location='Lower Manhattan', product_id='32', unit_price='3', product_category='Coffee', product_type='Gourmet brewed coffee', product_detail='Ethiopia Rg'),
 Row(transaction_id='2', transaction_date='1/1/2023', transaction_time='7:08:56', transaction_qty='2', store_id='5', store_location='Lower Manhattan', product_id='57', unit_price='3.1', product_category='Tea', product_type='Brewed Chai tea', product_detail='Spicy Eye Opener Chai Lg'),
 Row(transaction_id='3', transaction_date='1/1/2023', transaction_time='7:14:04', transaction_qty='2', store_id='5', store_location='Lower Manhattan', product_id='59', unit_price='4.5', product_category='Drinking Chocolate', product_type='Hot chocolate', product_detail='Dark chocolate Lg'),
 Row(transaction_id='4', transaction_date='1/1/2023', transaction_time='7:20:24', transaction_qty='1', store_id='5', store_location='Lower Man

In [3]:
# Load & Check the weather DataFrames
df_weather = spark.read.csv("./data/NYC.csv", header=True)
df_weather.head(5)

[Row(station='NYC', valid='2023-01-01 00:06', tmpc='10.00', relh='92.80', sped='0.00', p01m='T'),
 Row(station='NYC', valid='2023-01-01 00:18', tmpc='10.00', relh='95.98', sped='3.45', p01m='T'),
 Row(station='NYC', valid='2023-01-01 00:31', tmpc='10.00', relh='92.80', sped='0.00', p01m='0.25'),
 Row(station='NYC', valid='2023-01-01 00:44', tmpc='10.00', relh='95.98', sped='0.00', p01m='0.51'),
 Row(station='NYC', valid='2023-01-01 00:49', tmpc='10.00', relh='93.50', sped='0.00', p01m='0.51')]

In [4]:
# Check if any null values exists for the sales DataFrames
df_sales_null_count = df_sales.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_sales.columns])
df_sales_null_count.show()

+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+
|transaction_id|transaction_date|transaction_time|transaction_qty|store_id|store_location|product_id|unit_price|product_category|product_type|product_detail|
+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+
|             0|               0|               0|              0|       0|             0|         0|         0|               0|           0|             0|
+--------------+----------------+----------------+---------------+--------+--------------+----------+----------+----------------+------------+--------------+



In [5]:
# Check if any null values exists for the weather DataFrames
df_weather_null_count = df_weather.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_weather.columns])
df_weather_null_count.show()

+-------+-----+----+----+----+----+
|station|valid|tmpc|relh|sped|p01m|
+-------+-----+----+----+----+----+
|      0|    0|   0|   0| 312|   0|
+-------+-----+----+----+----+----+



In [6]:
# fill the wind speed null filed with 0
df_weather = df_weather.fillna({"sped": 0})

# There are some trace reports annotated with T, lets replace it with 0
df_weather = df_weather.withColumn(
    "p01m",
    when(col("p01m") == 'T', 0.00).otherwise(col("p01m"))
)

In [7]:
# Dropping some columns that is not required for the analysis
df_sales.drop("transaction_id")

DataFrame[transaction_date: string, transaction_time: string, transaction_qty: string, store_id: string, store_location: string, product_id: string, unit_price: string, product_category: string, product_type: string, product_detail: string]

In [8]:
# add new column date, time & hour for the sales DataFrames
df_sales = df_sales.withColumn("date", to_date("transaction_date", "M/d/yyyy")).drop("transaction_date")

df_sales = df_sales.withColumn("time", to_timestamp("transaction_time", "H:mm:ss")).drop("transaction_time")

df_sales = df_sales.withColumn("hour", hour("time")).drop("time")

In [9]:
df_sales.head(5)

[Row(transaction_id='1', transaction_qty='2', store_id='5', store_location='Lower Manhattan', product_id='32', unit_price='3', product_category='Coffee', product_type='Gourmet brewed coffee', product_detail='Ethiopia Rg', date=datetime.date(2023, 1, 1), hour=7),
 Row(transaction_id='2', transaction_qty='2', store_id='5', store_location='Lower Manhattan', product_id='57', unit_price='3.1', product_category='Tea', product_type='Brewed Chai tea', product_detail='Spicy Eye Opener Chai Lg', date=datetime.date(2023, 1, 1), hour=7),
 Row(transaction_id='3', transaction_qty='2', store_id='5', store_location='Lower Manhattan', product_id='59', unit_price='4.5', product_category='Drinking Chocolate', product_type='Hot chocolate', product_detail='Dark chocolate Lg', date=datetime.date(2023, 1, 1), hour=7),
 Row(transaction_id='4', transaction_qty='1', store_id='5', store_location='Lower Manhattan', product_id='22', unit_price='2', product_category='Coffee', product_type='Drip coffee', product_det

In [10]:
# add new column date, time & hour for the weather DataFrames
df_weather = df_weather.withColumn("timestamp", to_timestamp("valid", "yyyy-MM-dd HH:mm")).drop("valid")

df_weather = df_weather.withColumn("date", to_date("timestamp"))

df_weather = df_weather.withColumn("hour", hour("timestamp")).drop("timestamp")

In [11]:
df_weather.head()

Row(station='NYC', tmpc='10.00', relh='92.80', sped='0.00', p01m='0.0', date=datetime.date(2023, 1, 1), hour=0)

In [13]:
# We will take a subset of sales data only total sales per hour basis.
df_sales_subset = df_sales.select("store_id", "product_id", "date", "hour", "unit_price")

df_sales_per_hour = df_sales_subset.groupBy("store_id", "product_id", "date", "hour").agg(
    sum("unit_price").alias("total_sales")
)

In [14]:
# We will also take hourly mean temperature, relative humidity
df_weather = df_weather.drop("station")
df_weather_per_hour = df_weather.groupBy("date", "hour").agg(
    avg("tmpc").alias("avg_tempc"),
    avg("relh").alias("avg_relh")
)

In [15]:
# join two dataframe b date and hour
df_sales_weather = df_sales_per_hour.join(
    df_weather_per_hour,
    (df_sales_per_hour.date == df_weather_per_hour.date) & (df_sales_per_hour.hour == df_weather_per_hour.hour),
    "left"
).select(
    df_sales_per_hour.store_id,
    df_sales_per_hour.product_id,
    df_sales_per_hour.total_sales,
    df_weather_per_hour.avg_tempc,
    df_weather_per_hour.avg_relh,
    df_sales_per_hour.hour,
    df_sales_per_hour.date
)

df_sales_weather = df_sales_weather.withColumn("day", date_format("date", "EEEE"))

In [16]:
df_sales_weather.write.mode("overwrite").parquet("./data/sales_weather/")